In [12]:

import os
import cv2
import tkinter as tk
from tkinter import simpledialog
import mediapipe as mp

def capture_and_process(num_images):
    # Initialize Mediapipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    
    # Create the CollectedData folder if it doesn't exist
    folder_name = "CollectedData"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    # OpenCV camera feed
    cap = cv2.VideoCapture(0)
    print("Press 's' to save the cropped hand image with label, or 'q' to quit.")
    
    image_count = 0
    
    while image_count < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
        # Convert the frame to RGB for Mediapipe processing
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        
        # If hand landmarks are detected, draw them on the frame
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Display the camera feed with landmarks
        cv2.imshow("Capture Image with Landmarks", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and image_count < num_images:  # Save cropped hand image
            # Ask for label through Tkinter dialog
            root = tk.Tk()
            root.withdraw()
            label = simpledialog.askstring("Input", "Enter the label for this image:")
            root.destroy()
            
            if label and results.multi_hand_landmarks:
                # Create directory for the label if it doesn't exist
                label_dir = os.path.join(folder_name, label)
                if not os.path.exists(label_dir):
                    os.makedirs(label_dir)
                
                for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    # Calculate the bounding box of the hand
                    img_h, img_w, _ = frame.shape
                    x_min, y_min = img_w, img_h
                    x_max, y_max = 0, 0
                    
                    for landmark in hand_landmarks.landmark:
                        x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                        x_min, y_min = min(x, x_min), min(y, y_min)
                        x_max, y_max = max(x, x_max), max(y, y_max)
                    
                    # Add padding to the bounding box
                    padding = 20
                    x_min = max(0, x_min - padding)
                    y_min = max(0, y_min - padding)
                    x_max = min(img_w, x_max + padding)
                    y_max = min(img_h, y_max + padding)
                    
                    # Crop the hand region
                    cropped_hand = frame[y_min:y_max, x_min:x_max]
                    
                    # Save the cropped hand image with incremented index
                    file_name = os.path.join(label_dir, f"{idx + 1}.jpg")
                    cv2.imwrite(file_name, cropped_hand)
                    print(f"Cropped hand image saved as {file_name}")
                    image_count += 1
                    
                    # Stop if we've reached the required number of images
                    if image_count >= num_images:
                        break
            else:
                print("No hand detected or label not provided.")
        
        elif key == ord('q'):  # Quit
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the function with the desired number of images
capture_and_process(num_images=100)  # For example, save 100 images


Press 's' to save the cropped hand image with label, or 'q' to quit.
Cropped hand image saved as CollectedData\i love you\1.jpg
Cropped hand image saved as CollectedData\i lover you\1.jpg
Cropped hand image saved as CollectedData\hello\1.jpg
Cropped hand image saved as CollectedData\hello\1.jpg
